The goal here is to load the SQLite DB that CYGTT uses and generate the document (`can_you_git_to_that/database_description.json`) that is relied upon by the RAG to generate and query the database.

Note that the output may not be detailed nor accurate enough to drop-in and utilize without some human editing.  At least, in the case for this specific database in this specific situation, I felt it necessary
to tweak the generated text to suit my needs.  Thus, if you choose to alter the DB here in CYGTT, you may find this notebook helpful in regenerating the `database_description.json`, or you might simply choose to edit the file by (human) hand.  That said, this notebook should prove useful if you are investigating how to approach generating SQL through an LLM chat.

H/T to Ryan Nguyen's <A HREF="https://levelup.gitconnected.com/sql-generator-how-i-build-ai-query-wizard-for-enterprise-scale-with-500-tables-fc290692632a">SQL Generator 2.0: How I build AI Query Wizard for Enterprise-Scale with 500+ Tables</A> for helpful context and prompts.


In [ ]:
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy import inspect

# Connect to the existing SQLite database
engine = create_engine('sqlite:///../output/gravitymonkey-can_you_git_to_that.db')

# Create a metadata object
metadata = MetaData()

# Reflect the database schema
metadata.reflect(bind=engine)

In [ ]:
# Reflect the database schema (as before)
metadata.reflect(bind=engine)

# Print out schema and relationships
schema = ""
for table_name, table in metadata.tables.items():
    schema += f"Table: {table_name}\n"
    schema += "  Columns:\n"
    for column in table.columns:
        schema += f"    - {column.name} ({column.type})\n"

    schema += "  Foreign Keys:\n"
    for fk in table.foreign_keys:
       schema += f"    - {fk.column.table.name}.{fk.column.name} -> {table_name}.{fk.parent.name}\n"
    
    schema += "------------------\n\n"

In [ ]:

setup = """
	You are an expert database developer, and you have a database 
	schema which holds data to help users understand the history 
	and development of a GitHub repository. The database organizes, 
	classifies, and visualizes data about a repository, ideally 
	providing insights into its contributors, types of work, 
	progression over time, significant changes, and overall 
	lifecycle. 

    Your task is to review the database schema and to generate 
	documentation in JSON format. Focus on providing descriptions 
	relevant to including table purposes, column descriptions, and 
	potential use cases. Be concise yet informative, and ensure 
	all output is in valid JSON format.


	Generate comprehensive documentation for the following database 
	schema in JSON FORMAT ONLY. The documentation should include:

	1. A brief overview of the table's purpose and its role in 
	providing information about the repository

	2. Detailed descriptions of each column, including its data 
	type, purpose, and any relevant notes specific to repo analysis

	3. Any additional insights, best practices, or potential use 
	cases for this table in the context of repo, code, or 
	contributor analysis

	4. Generate a few common queries that could be run against 
	this table to extract meaningful insights

	Here's the schema:

"""

output = """
	Please provide the output in the following format:
	```json
	{{
      "TableName": "Name of the table",
      "TableDescription": "Brief overview of the table",
      "Columns": [
        {{
          "name": "column_name",
          "type": "data_type",
          "description": "Detailed description and purpose of the column"
        }},
        // ... all other columns
      ],
      "AdditionalInsights": [
        "Insight 1",
        "Insight 2",
        // ... other insights
      ],
      "CommonQueries": [
            {
                "natural_language": "Nature english query",
                "sql_query": "Detail of SQL query",
            }
      ]
    }}
    ```
    """



In [ ]:
from openai import OpenAI

client = OpenAI(
    api_key='XXXX', 
)

messages = [
    {
        "role": "user",
        "content": setup + schema + output,
    }
]

response = client.chat.completions.create(
	model="gpt-4o",
	messages=messages,
)
print(response.choices[0].message.content)